In [1]:
import pandas as pd
import numpy as np
import os
print("import done")
import requests
import json 
import csv 

import done


In [2]:
#getjson data from url and storing it on local system

url = 'https://api.covid19india.org/raw_data.json'

data = requests.get(url=url).json()
data = json.dumps(
    data,
    indent=2,
    ensure_ascii=False,
)

f = open('data/raw.json', 'w')
f.write(data)
f.close()

In [3]:
# JSON file to CSV 
# Opening JSON file and loading the data 
# into the variable data 

with open('data/raw.json') as json_file: 
    data = json.load(json_file) 

patient_data = data['raw_data'] 

# now we will open a file for writing 
data_file = open('data/data_file.csv', 'w') 

# create the csv writer object 
csv_writer = csv.writer(data_file) 

# Counter variable used for writing 
# headers to the CSV file 
count = 0

for pt in patient_data: 
    if count == 0: 

        # Writing headers of CSV file 
        header = pt.keys() 
        csv_writer.writerow(header) 
        count += 1

    # Writing data of CSV file 
    csv_writer.writerow(pt.values()) 

data_file.close() 
print("done")

done


In [4]:
#importing row data 
#This is Indian data for COVID-19 include all infomation of the infected person
raw_data  = pd.read_csv(r"data\data_file.csv",sep=",",encoding='cp1252')
data= pd.DataFrame(raw_data)
Column_Name=data.columns
Column_Name

#extract usefull column and create a dataframe

Indian_data=data[['patientnumber', 'dateannounced',
        'agebracket', 'gender','detectedstate', 'currentstatus',
       'typeoftransmission','statuschangedate']]
#renaming the column name
Indian_data = Indian_data.rename({"patientnumber": "PatientNumber", "dateannounced" :"InfectionDate",
        "agebracket":"Age","detectedstate":"State", "currentstatus":"Status",
       "typeoftransmission":"TransmissionType","statuschangedate":"StatusChangeDate",'gender':"Gender"}, axis=1)
cols= Indian_data.columns

#removing NA values
Indian_data = Indian_data.loc[:, cols].fillna("-")
Indian_data


,PatientNumber,InfectionDate,Age,Gender,State,Status,TransmissionType,StatusChangeDate
0,1,30/01/2020,20,F,Kerala,Recovered,Imported,14/02/2020
1,2,02/02/2020,-,-,Kerala,Recovered,Imported,14/02/2020
2,3,03/02/2020,-,-,Kerala,Recovered,Imported,14/02/2020
3,4,02/03/2020,45,M,Delhi,Recovered,Imported,15/03/2020
4,5,02/03/2020,24,M,Telangana,Recovered,Imported,02/03/2020
...,...,...,...,...,...,...,...,...
20077,-,21/04/2020,-,-,Rajasthan,Hospitalized,-,-
20078,-,21/04/2020,-,-,Rajasthan,Hospitalized,-,-
20079,-,21/04/2020,-,-,Rajasthan,Hospitalized,-,-
20080,-,21/04/2020,-,-,Rajasthan,Hospitalized,-,-


In [5]:
# Rate of Infection

RateofInfection= pd.DataFrame(Indian_data["InfectionDate"])
RateofInfection=RateofInfection["InfectionDate"].value_counts()

# Gender wise Infection 

GenderWiseINfection= pd.DataFrame(Indian_data["Gender"])
GenderWiseINfection=GenderWiseINfection["Gender"].value_counts()
GenderWiseINfection

#State wise Infection

StateWiseInfection= pd.DataFrame(Indian_data['State'])
StateWiseInfection=StateWiseInfection["State"].value_counts()
StateWiseInfection
RateofInfection['13/04/2020']

#Current Status of Patient
CurrentPatientStatus=pd.DataFrame(Indian_data['Status'])
CurrentPatientStatus=CurrentPatientStatus["Status"].value_counts()
CurrentPatientStatus

Hospitalized    19855
Recovered         181
Deceased           45
Migrated            1
Name: Status, dtype: int64

In [8]:
StateWiseROIperDay=Indian_data[['InfectionDate','State']]
StateWiseROIperDay=StateWiseROIperDay.groupby(['InfectionDate', 'State']).size().reset_index(name="Time")
StateWiseROIperDay
StateWiseInfection= pd.DataFrame(Indian_data['State'])
StateWiseInfection=pd.DataFrame(StateWiseInfection["State"].value_counts())
StateWiseInfection=StateWiseInfection.reset_index()
StateWiseInfection=StateWiseInfection.rename({"index":"State","State":"Count"},axis=1)
StateWiseInfection=StateWiseInfection.sort_values(by='State')
StateWiseInfection=StateWiseInfection.iloc[1:]
StateWiseInfection

,State,Count
24,Andaman and Nicobar Islands,17
8,Andhra Pradesh,757
31,Arunachal Pradesh,1
21,Assam,35
15,Bihar,126
22,Chandigarh,27
20,Chhattisgarh,36
2,Delhi,2156
27,Goa,7
1,Gujarat,2178


In [9]:
Indina_state={
    "AN":"Andaman and Nicobar Islands",
    "AP":"Andhra Pradesh",
    "AR":"Arunachal Pradesh",
    "AS":"Assam",
    "BR":"Bihar",
    "CG":"Chandigarh",
    "CH":"Chhattisgarh",
    "DN":"Dadra and Nagar Haveli",
    "DD":"Daman and Diu",
    "DL":"Delhi",
    "GA":"Goa",
    "GJ":"Gujarat",
    "HR":"Haryana",
    "HP":"Himachal Pradesh",
    "JK":"Jammu and Kashmir",
    "JH":"Jharkhand",
    "KA":"Karnataka",
    "KL":"Kerala",
    "LA":"Ladakh",
    "LD":"Lakshadweep",
    "MP":"Madhya Pradesh",
    "MH":"Maharashtra",
    "MN":"Manipur",
        "ML":"Meghalaya",
    "MZ":"Mizoram",
    "NL":"Nagaland",
    "OR":"Odisha",
    "PY":"Puducherry",
    "PB":"Punjab",
    "RJ":"Rajasthan",
    "SK":"Sikkim",
    "TN":"Tamil Nadu",
    "TS":"Telangana",
    "TR":"Tripura",
    "UP":"Uttar Pradesh",
    "UK":"Uttarakhand",
    "WB":"West Bengal"
}

state = {s:a for a,s in Indina_state.items()}
state["Andhra Pradesh"]


'AP'

In [11]:
India=pd.DataFrame(pd.read_csv('data/in.csv'))
India_state=India.loc[India['capital'] == 'admin']
col=['admin','city', 'lat', 'lng']
India_state=India_state[col]
India_state=India_state.rename({'admin':'State'},axis=1)
India_state=India_state.reset_index()
del India_state['index']
India_state=India_state.sort_values(by='State')
India_state
#India_state['State'] = India_state['State'].apply(lambda s: state.get(s))
India_state

,State,city,lat,lng
26,Andaman and Nicobar Islands,Port Blair,11.666667,92.750000
8,Andhra Pradesh,Amravati,16.566664,80.366665
30,Arunachal Pradesh,Itanagar,27.102349,93.692047
33,Assam,Dispur,26.135638,91.800688
10,Bihar,Patna,25.615379,85.101027
15,Chandigarh,Chandigarh,30.741482,76.768066
17,Chhattisgarh,Raipur,21.233333,81.633333
31,Damān and Diu,Daman,20.414315,72.832360
3,Delhi,Delhi,28.651952,77.231495
32,Dādra and Nagar Haveli,Silvassa,20.273855,72.996728


In [12]:
StateWiseInfection1= StateWiseInfection.merge(India_state,on="State",how='left')
StateWiseInfection1

,State,Count,city,lat,lng
0,Andaman and Nicobar Islands,17,Port Blair,11.666667,92.750000
1,Andhra Pradesh,757,Amravati,16.566664,80.366665
2,Arunachal Pradesh,1,Itanagar,27.102349,93.692047
3,Assam,35,Dispur,26.135638,91.800688
4,Bihar,126,Patna,25.615379,85.101027
5,Chandigarh,27,Chandigarh,30.741482,76.768066
6,Chhattisgarh,36,Raipur,21.233333,81.633333
7,Delhi,2156,Delhi,28.651952,77.231495
8,Goa,7,Panaji,15.498289,73.824541
9,Gujarat,2178,Ghandinagar,23.216667,72.683333
